<a href="https://colab.research.google.com/github/ole12345/yolov9/blob/main/yolo_v9train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/ole12345/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 555, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 555 (delta 24), reused 46 (delta 20), pack-reused 499 (from 1)
Receiving objects: 100% (555/555), 2.63 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (299/299), done.


In [ ]:
%cd yolov9
!pip install -r requirements.txt
import os
os.kill(os.getpid(), 9)


/content/yolov9
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124

In [1]:
%cd yolov9
!pwd

/content/yolov9
/content/yolov9


In [2]:
!pip install --upgrade jax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 MB 23.5 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
    Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
      Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89
  Attempting uninstall: jax
    Found existing installation: jax 0.4.26
    Uninstalling jax-0.4.26:
      Successfully uninstalled jax-0.4.26


In [3]:
import os
from utils.filter_dataset import DatasetConfig, SubsetStrEnum, filter_dataset
import yaml

root_dir = os.getcwd()
dataset_name="coco-2017-person-ball-small"
category_list=["person","sports ball"]
yaml_content = {"path":os.path.join(".","datasets",dataset_name)}

#validation
subset = SubsetStrEnum.VALIDATION
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_val = filter_dataset(cfg)
yaml_content["val"] = subset+".txt"
#train
subset = SubsetStrEnum.TRAIN
max_number_of_images = None
cfg= DatasetConfig(root_dir = root_dir,dataset_name=dataset_name, subset = subset, max_number_of_images = max_number_of_images,category_list=category_list)
cat_id_name_train = filter_dataset(cfg)
yaml_content["train"] = subset+".txt"
if cat_id_name_train!=cat_id_name_val:
   raise Exception("Inconsistent categories")

#write the yaml file
yaml_content["names"] = cat_id_name_train
yaml_content["stuff_names"] = list()
yaml_content["download"] = None
with open(os.path.join(root_dir,"data",dataset_name+".yaml"),"w") as file:
   yaml.dump(yaml_content, file, default_flow_style=False)
   file.close()

annotations_trainval2017.zip: 100%|██████████| 3.77M/3.77M [00:31<00:00, 8.00MiB/s]


loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
Selected 2522 images of 'instances_val2017.json' with following class distribution:
'      person' : 2512 images
' sports ball' : 169 images


New Category Ids:
0:person; old id: 1
1:sports ball; old id: 37
loading annotations into memory...
Done (t=15.89s)
creating index...
index created!
Selected 59618 images of 'instances_train2017.json' with following class distribution:
'      person' : 59453 images
' sports ball' : 4262 images


New Category Ids:
0:person; old id: 1
1:sports ball; old id: 37


In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Sep 22 09:16:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   38C    P8              11W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from segment.train import run

run(data = "coco-2017-person-ball-small.yaml",
    device = 0,
    batch = 32,
    img = 640,
    cfg = "models/segment/gelan-c-seg-test.yaml",
    weights = "/content/drive/My Drive/test_model2/weights/last.pt",
    name = "test_model",
    hyp = "hyp.scratch-high.yaml",
    close_mosaic = 10,
    epochs = 10,
    project = "/content/drive/My Drive"
)

segment/train: weights=/content/drive/My Drive/test_model2/weights/last.pt, cfg=models/segment/gelan-c-seg-test.yaml, data=coco-2017-person-ball-small.yaml, hyp=hyp.scratch-high.yaml, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/My Drive, name=test_model, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, close_mosaic=10, mask_ratio=4, no_overlap=False, batch=32, img=640
YOLO 🚀 778de8d Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=7.5, cls=0.5, cls_pw=1.0, obj=0.7, obj_pw=1.0, dfl=1.5, iou_t=0.2, anchor_t=5.0, fl_gam

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.55s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=2.85s).
Accumulating evaluation results...
DONE (t=1.88s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.002
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.052
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

Results saved to /content/drive/My Drive/test_model3


Namespace(weights='/content/drive/My Drive/test_model2/weights/last.pt', cfg='models/segment/gelan-c-seg-test.yaml', data='/content/yolov9/data/coco-2017-person-ball-small.yaml', hyp={'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'warmup_bias_lr': 0.1, 'box': 7.5, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 0.7, 'obj_pw': 1.0, 'dfl': 1.5, 'iou_t': 0.2, 'anchor_t': 5.0, 'fl_gamma': 0.0, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.9, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.15, 'copy_paste': 0.3}, epochs=10, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket='', cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer='SGD', sync_bn=False, workers=8, project='/content/drive/My Drive', name='test_model', exist_ok=False, quad

In [6]:
!python export.py --weights /content/drive/My Drive/test_model3/weights/best.pt --include onnx

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968